In [ ]:
import numpy as np
import numpy.matlib
import matplotlib.pyplot as plt

%matplotlib inline

## Discrete updating algorithm

### Continuous time definitions
$y_{\textrm{ref}}$ : Reference level

$y_{\textrm{SAM}}$ : Output of SAM

$y_{\textrm{MPM}}$ : Output of of MPM

$I_{\textrm{SAM}}$ : Input to SAM

$I_{\textrm{MPM}}$ : Input to MPM


### Translation into discrete time
$i$ will index *time-points* and *intervals*

$t_p^i$ : time of $i$th production

$t_s^i$ : time of $i$th stimulus

$\textrm{ISI}^i$ = $t_s^{i+1} - t_s^i$

$\textrm{IPI}^i$ = $t_p^{i+1} - t_p^i$

$y_{\textrm{SAM}}^i=I_{\textrm{SAM}}^i*\textrm{ISI}^i$

$I_{\textrm{SAM}}^{i+1}=I_{\textrm{SAM}}^i+\beta(y_{\textrm{ref}}-y_{\textrm{SAM}}^i)$

$I_{\textrm{MPM}}^{i+1} = I_{\textrm{SAM}}^{i+1} + \alpha(y_{\textrm{out}}^i-y_{\textrm{MPM}}^i)$

$\textrm{IPI}^i=y_{\textrm{ref}}/I_{\textrm{MPM}}$

$t_p^{i+1}=t_p^i+\textrm{IPI}^i$

By definition $y_{\textrm{MPM}}^i=y_{\textrm{ref}}$ (output of $y_{\textrm{MPM}}$ at ith production)

The value of $y_{\textrm{out}}^i$ represents the output of the SAM at the time of the $i$th production. During synchronization that will be:

$y_{\textrm{out}}^i = I_{\textrm{SAM}}^i*(t_p^i-t_s^{i-1})$

During continuation:

$y_{\textrm{out}}^i = y_{\textrm{SAM}}^\infty$

with $y_{\textrm{SAM}}^\infty$ representing the terminal output of the SAM module.

In [ ]:
# Definitions
def I_s_update(I_s_in,beta,y_ref,y_s):
    I_s_out = I_s_in + beta*(y_ref - y_s)
    
    return I_s_out

def I_p_update(I_s,alpha,y_p,y_out):
    I_p_out = I_s + alpha*(y_out-y_p)
    
    return I_p_out

def discreteAlgorithm_multiple(ISI, n, beta=0.5, alpha=0.1, y_ref=1, 
                               I_s_init=0.8, y_s_infty=1.5, sigma=0):
    '''Perform multiple simulations of the discrete algorithm
    Returns: an array of size n x T'''
    IPIs = []
    for i in range(n):
        _, t_p, IPI, _, _, _ = discreteAlgorithm(ISI, beta, alpha, y_ref, I_s_init,
                                                        y_s_infty, sigma)
        #print('tp and IPI')
        #print(t_p)
        #print(IPI)
        #print(np.diff(t_p))
        IPI_noise = np.diff(t_p)
        IPIs.append(IPI_noise)
        
    return np.array(IPIs)
    


def discreteAlgorithm(ISI, beta=0.5, alpha=0.1, y_ref=1, I_s_init=0.8, y_s_infty=1.5, sigma=0):

    t_s = np.zeros(len(ISI)+1)
    t_s[1:len(ISI)+1] = np.cumsum(ISI)
    IPI = np.zeros((len(t_s))-1)
    t_p = np.zeros((len(t_s)))
    I_s = np.zeros((len(t_s)))
    I_p = np.zeros((len(t_s)))
    y_s = np.zeros((len(t_s)))
    I_p[0] = I_s_init
    I_s[0] = I_s_init

    for i in range(len(ISI)):
        
        IPI[i] = y_ref/I_p[i]
        if i > 0:
            noise = np.random.randn() * sigma
            
            t_p[i+1] = t_p[i] + IPI[i] + noise
            #print(t_p)
            #print(IPI)
        else:
            #print(i, 'here2')
            t_p[i+1] = IPI[i] + np.random.randn() * sigma
        
        if np.isnan(ISI[i]):
            y_s[i] = y_s_infty
            I_s[i+1] = I_s[i]
        else:
            y_s[i] = I_s[i]*ISI[i]
            I_s[i+1] = I_s_update(I_s[i],beta,y_ref,y_s[i])

        
        if np.isnan(ISI[i]):
            y_out = y_s_infty
        else:
            if i == 0:
                y_out = y_ref
            else:
                y_out = I_s[i]*(t_p[i]-t_s[i-1])
            #print(i,y_out)

        # Standard code for algorithm, implementing integration of differences over time
        I_p[i+1] = I_p_update(I_s[i+1],alpha,y_ref,y_out)
        
        # Code that does not integrate differences over time
        #if np.isnan(t_s[i]):
        #    I_p[i+1] = I_p_update(I_s[i+1],alpha,t_p[i],t_p[i]+0.4)
        #else:       
        #    I_p[i+1] = I_p_update(I_s[i+1],alpha,t_p[i],t_s[i])
        
        #print(t_s[i],t_p[i],IPI[i])
    #print(t_p)
    return t_s, t_p, IPI, I_s, I_p, y_s

In [ ]:
discreteAlgorithm_multiple([0.5] * 4 + [np.nan] * 7, 3, sigma=0.1)

## Synchronization to a constant ISI

In [ ]:
# Simple synchronization
ISI = [0.4] * 500
t_s, t_p, IPI, I_s, I_p, y_s = discreteAlgorithm(ISI, beta=0.2, alpha=0.1, y_ref=1, I_s_init=1/0.35, sigma=0)

#for i in range(len(t_s)):
#    print(t_s[i],t_p[i],IPI[i])

In [ ]:
plt.plot(ISI)
plt.plot(IPI)

In [ ]:
plt.plot(t_p-t_s)

## Step Change

In [ ]:
# Step change
ISI = np.concatenate((0.80*np.ones(100),1.00*np.ones(30)))
del t_p
del t_s
del IPI
del I_s
del I_p
del y_s
t_s, t_p, IPI, I_s, I_p, y_s = discreteAlgorithm(ISI, beta=0.5, alpha=0.1, y_ref=1, I_s_init=1/0.8, sigma=0)

In [ ]:
plt.plot(ISI)
plt.plot(IPI)

In [ ]:
plt.plot(t_p-t_s)

In [ ]:
ISI = np.concatenate((0.80*np.ones(4),1.00*np.ones(9)))
betas = [0.5,0.7,0.9]
alpha = 0.3
TS = np.zeros((14,len(betas)))
TP = np.zeros((14,len(betas)))
IPIs = np.zeros((13,len(betas)))
for betai in range(len(betas)):
    t_s, t_p, IPI, I_s, I_p, y_s = discreteAlgorithm(ISI, beta=betas[betai], alpha=alpha, y_ref=1, I_s_init=1/0.8)
    TS[:,betai] = t_s
    TP[:,betai] = t_p
    IPIs[:,betai] = IPI
    

In [ ]:
print(ISI,IPIs[0:-1,0])
plt.plot(ISI)
plt.plot(IPIs[0:-1,:])

In [ ]:
plt.plot(TP-TS)

## Phase shift

In [ ]:
# Phase shift
ISI = np.concatenate((0.80*np.ones(50),0.9*np.ones(1),0.80*np.ones(50)))
del t_p
del t_s
del IPI
del I_s
del I_p
del y_s
t_s, t_p, IPI, I_s, I_p, y_s = discreteAlgorithm(ISI, beta=0.7, alpha=0.1, y_ref=1, I_s_init=1/0.8, sigma=0)

In [ ]:
plt.plot(ISI)
plt.plot(IPI)

In [ ]:
plt.plot(t_p-t_s)

In [ ]:
ISI = np.concatenate((0.80*np.ones(4),0.9*np.ones(1),0.80*np.ones(8)))
beta = 0.7
alphas = [0.1,0.2,0.3]
TS = np.zeros((14,len(betas)))
TP = np.zeros((14,len(betas)))
IPIs = np.zeros((13,len(betas)))
for alphai in range(len(alphas)):
    t_s, t_p, IPI, I_s, I_p, y_s = discreteAlgorithm(ISI, beta=beta, alpha=alphas[alphai], y_ref=1, I_s_init=1/0.8)
    TS[:,alphai] = t_s
    TP[:,alphai] = t_p
    IPIs[:,alphai] = IPI

In [ ]:
plt.plot(ISI)
plt.plot(IPIs)

In [ ]:
plt.plot(TP-TS)

## Stimulus jitter

In [ ]:
# Stimulus jitter
ISI = np.concatenate((0.80*np.ones(50),0.9*np.ones(1), 0.7*np.ones(1), 0.80*np.ones(49)))
del t_p
del t_s
del IPI
del I_s
del I_p
del y_s
t_s, t_p, IPI, I_s, I_p, y_s = discreteAlgorithm(ISI, beta=0.7, alpha=0.1, y_ref=1, I_s_init=1/0.8, sigma=0)

In [ ]:
plt.plot(ISI)
plt.plot(IPI)

In [ ]:
plt.plot(t_p-t_s)

In [ ]:
ISI = np.concatenate((0.80*np.ones(4),0.9*np.ones(1), 0.7*np.ones(1), 0.80*np.ones(7)))
beta = 0.7
alphas = [0.1,0.2,0.3]
TS = np.zeros((14,len(betas)))
TP = np.zeros((14,len(betas)))
IPIs = np.zeros((13,len(betas)))
for alphai in range(len(alphas)):
    t_s, t_p, IPI, I_s, I_p, y_s = discreteAlgorithm(ISI, beta=beta, alpha=alphas[alphai], y_ref=1, I_s_init=1/0.8)
    TS[:,alphai] = t_s
    TP[:,alphai] = t_p
    IPIs[:,alphai] = IPI

In [ ]:
plt.plot(ISI)
plt.plot(IPIs)

In [ ]:
plt.plot(TP-TS)

## 1-2-Go, 1-2-3-Go

In [ ]:
def simulate_123Go(nstages=3,beta=0.75,alpha=0.0,ISI0=0.8,y_ref=1,ISImin=0.600,
                   ISImax=1.000,number=5, sigma=0):
    # 1-2-Go, 1-2-3-Go
    ISIs = np.linspace(ISImin,ISImax,num=number)
    ISI_list = np.nan*np.ones((len(ISIs),nstages))
    IPI_list = np.nan*np.ones((len(ISIs),nstages))
    t_s_list = []
    t_p_list = []
    for i in range(len(ISIs)):
        ISI_list[i,:] = ISIs[i]*np.ones(nstages)
        ISI = ISI_list[i,:]
        t_s, t_p, IPI, I_s, I_p, y_s = discreteAlgorithm(ISI, beta=beta, alpha=alpha, 
                                                         y_ref=y_ref, I_s_init=1/ISI0,
                                                        sigma=sigma)

        t_s_list.append(t_s)
        t_p_list.append(t_p)
        IPI_list[i,:] = IPI
    
    return IPI_list, ISIs


def simulate_123Go_noise(nstages=3,beta=0.75,alpha=0.0,ISI0=0.8,y_ref=1,ISImin=0.600,
                   ISImax=1.000,number=5, sigma=0, ntrials=100):
    # 1-2-Go, 1-2-3-Go
    ISIs = np.linspace(ISImin,ISImax,num=number)
    meanIPIs = []
    stdIPIs = []
    
    for i in range(len(ISIs)): 
        ISI = ISIs[i]*np.ones(nstages)
        IPIs = discreteAlgorithm_multiple(ISI, ntrials,
                                    beta=beta, alpha=alpha, 
                                    y_ref=y_ref, I_s_init=1/ISI0,
                                    sigma=sigma)[:,-1]
        meanIPIs.append(np.mean(IPIs))
        stdIPIs.append(np.std(IPIs))

    
    return np.array(meanIPIs), np.array(stdIPIs)


In [ ]:
print(IPI_list.shape)
tempISIs = np.matlib.repmat(ISIs,3,1)
sqErrs = np.square(IPI_list - np.transpose(tempISIs))
sqBIAS = sqErrs.sum(axis=0)/len(ISIs)
print(sqBIAS)

In [ ]:
plt.subplot(121)
plt.plot(sqBIAS[1:3])

plt.subplot(122)
plt.plot(np.transpose(tempISIs),IPI_list)

## Synchronization/continuation

In [ ]:
# Synchronization/continuation
ISIs = np.linspace(0.550,0.817,num=5)
print(ISIs)
ISI_list = np.nan*np.ones((len(ISIs),12))
IPI_list = np.nan*np.ones((len(ISIs),12))
t_s_list = []
t_p_list = []
for i in range(len(ISIs)):
    ISI_list[i,:] = np.concatenate((ISIs[i]*np.ones(4),np.nan*np.ones(8)))
    del t_p
    del t_s
    del IPI
    del I_s
    del I_p
    del y_s
    ISI = ISI_list[i,:]
    t_s, t_p, IPI, I_s, I_p, y_s = discreteAlgorithm(ISI, beta=0.75, alpha=0.1, y_ref=1, 
                                                     I_s_init=1/0.6835, y_s_infty=1.5,
                                                    sigma=0)
    
    t_s_list.append(t_s)
    t_p_list.append(t_p)
    IPI_list[i,:] = IPI

In [ ]:
plt.plot(ISI)
plt.plot(IPI)

In [ ]:
plt.plot(t_p-t_s)

In [ ]:
tempISIs = np.matlib.repmat(ISIs,12,1)
sqErrs = np.square(IPI_list - np.transpose(tempISIs))
sqBIAS = sqErrs.sum(axis=0)/len(ISIs)
print(sqBIAS)

In [ ]:
plt.subplot(121)
plt.plot(sqBIAS[1:12])

plt.subplot(122)
print(tempISIs.shape)
plt.plot(np.transpose(tempISIs[(0,4,10),:]),IPI_list[:,(0,4,10)])

### Fit to behavior

#### Synchronization/continuation

In [ ]:
import scipy.io
import glob

def find_bias_arr(filename, tmax):
    '''filename: SynCon_ITIs file name
    tmax: max number of taps to consider
    
    Returns: the bias array, tmax x number of intervals in prior'''
    data = scipy.io.loadmat(filename)
    durs = data['durs'].flatten()
    splits = data['allDur_splits']
    IPI_mean = data['allDur_mean'][0]
    
    Bias = np.zeros((tmax, 5))

    for i in range(len(IPI_mean)):
        IPI_single = IPI_mean[i].flatten()[:tmax]
        bias_single = IPI_single - durs[i]
        Bias[:,i] = bias_single
    
    return Bias

def plot_subject_ts_tp(filename):
    bias_arr = find_bias_arr(filename, 20)
    std_arr = find_std_arr(filename, 20)
    data = scipy.io.loadmat(filename)
    durs = data['durs'].flatten()

    # Plot ts-tp dependence (subject)
    bias_first = bias_arr[0,:]
    std_first = std_arr[0,:]
    bias_third = bias_arr[2,:]
    std_third = std_arr[2,:]
    bias_fifth = bias_arr[4,:]
    std_fifth = std_arr[4,:]

    plt.errorbar(durs, durs + bias_third, std_third)
    plt.errorbar(durs, durs + bias_fifth, std_fifth)
    plt.plot(durs, durs, '--')

def find_std_arr(filename, tmax):
    '''filename: SynCon_ITIs file name
    tmax: max number of taps to consider
    
    Returns: the bias array, tmax x number of intervals in prior'''
    data = scipy.io.loadmat(filename)
    durs = data['durs'].flatten()
    splits = data['allDur_splits']
    IPI_std = data['allDur_std'][0]
    
    STD = np.zeros((tmax, 5))

    for i in range(len(IPI_std)):
        std_single = IPI_std[i].flatten()[:tmax]
        STD[:,i] = std_single
    
    return STD
    
def simulate_sync_cont_bias(ISIs, ntrials, beta, alpha, ISI0, sigma):
    meanITIs = []

    for i in range(len(ISIs)):
        ISI_list = np.concatenate((ISIs[i]*np.ones(4),np.nan*np.ones(17)))        
        IPI_lst = discreteAlgorithm_multiple(ISI_list, ntrials, beta, alpha, y_ref=1, 
                               I_s_init=ISI0, y_s_infty=1.5, sigma=sigma)
        #print(IPI_lst)
        meanIPI = np.mean(IPI_lst, axis=0)
        meanITIs.append(meanIPI)
        
    meanITIs = np.array(meanITIs)
    bias_individual = meanITIs - ISIs[:,np.newaxis]
    bias_all_sim = np.mean(bias_individual**2, axis=0)
    return np.sqrt(bias_all_sim)[1:]

def minimizer_noise(filename,ISIs,nsamples,betaRange,alphaRange,ISI0Range,sigma):
    bias_arr = find_bias_arr(filename, 20)
    betas = np.random.uniform(low=betaRange[0], high=betaRange[1], size=nsamples)
    alphas = np.random.uniform(low=alphaRange[0], high=alphaRange[1], size=nsamples)
    ISI0s = np.random.uniform(low=ISI0Range[0], high=ISI0Range[1], size=nsamples)
    mse = np.nan*np.ones((nsamples,1))
    
    ntrials = 21
    for j in range(nsamples):
        biasSim = simulate_sync_cont_bias(ISIs, ntrials, betas[j], 
                                          alphas[j], 1/ISI0s[j], sigma)
        biasObs = np.sqrt( np.nanmean(bias_arr**2,axis=1) )
        mse[j] = np.nanmean((biasObs - biasSim)**2)
        
    id = np.argmin(mse)
    print(id)
    beta = betas[id]
    alpha = alphas[id]
    ISI0 = ISI0s[id]
    return beta, alpha, ISI0

def minimizer(filename,ISIs,nsamples,betaRange,alphaRange,ISI0Range):
    bias_arr = find_bias_arr(filename, 20)
    
    betas = np.random.uniform(low=betaRange[0], high=betaRange[1], size=nsamples)
    alphas = np.random.uniform(low=alphaRange[0], high=alphaRange[1], size=nsamples)
    ISI0s = np.random.uniform(low=ISI0Range[0], high=ISI0Range[1], size=nsamples)
    mse = np.nan*np.ones((nsamples,1))
    for j in range(nsamples):
        ISI_list = np.nan*np.ones((len(ISIs),21))
        ISI_list2 = np.nan*np.ones((len(ISIs),21))
        IPI_list = np.nan*np.ones((len(ISIs),21))
        t_s_list = []
        t_p_list = []
        for i in range(len(ISIs)):
            ISI_list[i,:] = np.concatenate((ISIs[i]*np.ones(4),np.nan*np.ones(17)))
            ISI_list2[i,:] = ISIs[i]*np.ones(21)
            ISI = ISI_list[i,:]
            _, _, IPI, _, _, _ = discreteAlgorithm(ISI, beta=betas[j], alpha=alphas[j], y_ref=1,
                                                             I_s_init=1/ISI0s[j], y_s_infty=1.5, sigma=0)
            IPI_list[i,:] = IPI
            
        biasSim = np.sqrt( np.nanmean((IPI_list[:,1:]-ISI_list2[:,1:])**2,axis=0) )
        biasObs = np.sqrt( np.nanmean(bias_arr**2,axis=1) )
        mse[j] = np.nanmean((biasObs - biasSim)**2)
           
    id = np.argmin(mse)
    print(id)
    beta = betas[id]
    alpha = alphas[id]
    ISI0 = ISI0s[id]
    return beta, alpha, ISI0

In [ ]:
ISIs = np.linspace(0.550,0.817,num=5)
dirlst = glob.glob('C:\\Users\\Le\\Dropbox (MIT)\\Jazayeri\\SyncContData\\*_2_*ITIs.mat')
#dirlst = glob.glob('/Users/swe/Dropbox (MIT)/CircuitModel/Data/*_2_*ITIs.mat')
filename = dirlst[4] #'/Users/swe/Dropbox (MIT)/CircuitModel/Data/FK_2_20170721_SynCon_ITIs.mat'
print(filename)
beta, alpha, ISI0 = minimizer(filename, ISIs,nsamples=1000, betaRange=[0,0.8], alphaRange=[0,0.8], ISI0Range=[0.5,0.9])


In [ ]:
BETAS = np.zeros((len(dirlst),1))
ALPHAS = np.zeros((len(dirlst),1))
ISIinits = np.zeros((len(dirlst),1))
for filei in range(len(dirlst)):
    filename = dirlst[filei] #'/Users/swe/Dropbox (MIT)/CircuitModel/Data/FK_2_20170721_SynCon_ITIs.mat'
    print(filename)
    beta, alpha, ISI0 = minimizer_noise(filename, ISIs,nsamples=1000, 
                                        betaRange=[0,0.8], alphaRange=[0,0.8], 
                                        ISI0Range=[0.5,0.9], sigma=0.06)
    
    BETAS[filei] = beta
    ALPHAS[filei] = alpha
    ISIinits[filei] = ISI0

With the fitted values, we now perform the simulations.

In [ ]:
plotlst = [0,3,4,5,6,7]

Bias_sim_mean = np.zeros((6, 20))



for idx, filei in enumerate(plotlst):
    # Perform the simulation
    ISI_list = np.nan*np.ones((len(ISIs),21))
    IPI_list = np.nan*np.ones((len(ISIs),21))
    t_s_list = []
    t_p_list = []
    
    beta = BETAS[filei, 0]
    alpha = ALPHAS[filei, 0]
    ISI0 = ISIinits[filei, 0]
    filename = dirlst[filei]


    bias_arr = find_bias_arr(filename, 20)
    biasObs = np.sqrt(np.nanmean(bias_arr**2,axis=1))


    biasModel = simulate_sync_cont_bias(ISIs, ntrials=21, beta=beta, 
                                        alpha=alpha, ISI0=1/ISI0, sigma=0.06)
    plt.subplot(121)
    plt.plot(biasModel)
    
    plt.subplot(122)
    plt.plot(biasObs)

    Bias_sim_mean[idx, :] = biasModel * 1000
    #plt.plot(np.transpose(tempISIs[(3,5),:]),IPI_list[:,(3,5)])
    #plot_subject_ts_tp(filename)

In [ ]:
#scipy.io.savemat('PlotTools/sync_cont_figure_algorithmic_circuit_012120c.mat', {'alphasummary': ALPHAS,
#                                                             'Isummary': ISIinits,
#                                                             'Ksummary': BETAS,
#                                                             'Bias_sim_mean': Bias_sim_mean})

In [ ]:
# For figure 8b & 8c
plotlst = [0,3,4,5,6,7]

meanITI_model = []
stdITI_model = []

ntrials = 21

for idx, filei in enumerate(plotlst):
    # Perform the simulation
    beta = BETAS[filei, 0]
    alpha = ALPHAS[filei, 0]
    ISI0 = ISIinits[filei, 0]
    filename = dirlst[filei]
    sigma = 0.06
    
    meanITIs = []
    stdITIs = []

    for i in range(len(ISIs)):
        ISI_list = np.concatenate((ISIs[i]*np.ones(4),np.nan*np.ones(17)))        
        IPI_lst = discreteAlgorithm_multiple(ISI_list, ntrials, beta, alpha, y_ref=1, 
                               I_s_init=1/ISI0, y_s_infty=1.5, sigma=sigma)
        #print(IPI_lst)
        meanIPI = np.mean(IPI_lst, axis=0)
        stdIPI = np.std(IPI_lst, axis=0)
        meanITIs.append(meanIPI)
        stdITIs.append(stdIPI)
        
    meanITI_model.append(np.array(meanITIs)[:,1:] * 1000)
    stdITI_model.append(np.array(stdITIs)[:,1:] * 1000)

In [ ]:
#scipy.io.savemat('PlotTools/subject_ts_tp_sync_cont_discreteAlgo_012220.mat',
#                {'durs': ISIs, 'meanITI_model': meanITI_model, 
#                 'stdITI_model': stdITI_model})

### 1-2-Go/1-2-3-Go

In [ ]:
def get_time_lst(nstages, beta, alpha, ISI0):
    '''Simulate and return the array of times of threshold crossing'''
    IPI_list, ISIs = simulate_123Go(nstages=nstages,beta=beta,alpha=alpha,ISI0=ISI0)
    times = 1000*IPI_list[:,-1]

    return times
    

def do_beta_alpha_ISI0_fitting(subject_file, nsamples, ntrials, betalow, betahigh, 
                               alphalow, alphahigh, ISI0low, ISI0high, sigma):
    '''Inputs:
    - subject_file: .mat file with the behavioral results of the subject
    - nsamples: number of random (beta,alpha,ISI0) combinations to sample
    - betalow, betahigh, alphalow, alphahigh, ISI0low, ISI0high: ranges of beta, alpha, and ISI0 for sampling
    
    Outputs:
    beta, alpha, ISI0: the optimal combination to minimize the mse between the mean simulated times
    and the mean behavioral times of the subject'''
    #print('sigmaval = ', sigmaval)
    betalst = np.random.uniform(low=betalow, high=betahigh, size=nsamples)
    alphalst = np.random.uniform(low=alphalow, high=alphahigh, size=nsamples)
    ISI0lst = np.random.uniform(low=ISI0low, high=ISI0high, size=nsamples)

    # Load data for subject SWE(RSG/RSSG, Seth provided)
    swedata = scipy.io.loadmat(subject_file)
    mtp_in = swedata['mtp_in']
    stdtp_in = swedata['stdtp_in']

    target_time1 = mtp_in[:,0]
    target_time2 = mtp_in[:,1]

    errorlst = []

    for k in range(nsamples):
        ISI0val = ISI0lst[k]
        betaval = betalst[k]
        alphaval = alphalst[k]

        #1,2,Go simulation
        prod_time_lst, _ = simulate_123Go_noise(nstages=2, ntrials=ntrials,
                                    beta=betaval, alpha=alphaval, ISI0=ISI0val,
                                               sigma=sigma)

        #1,2,3,Go simulation
        prod_time_lst2, _ = simulate_123Go_noise(nstages=3, ntrials=ntrials,
                                    beta=betaval, alpha=alphaval, ISI0=ISI0val,
                                                sigma=sigma)

        # Continue if there is a nan   
        error1 = (np.transpose(prod_time_lst) * 1000 - target_time1) ** 2
        error2 = (np.transpose(prod_time_lst2) * 1000 - target_time2) ** 2
        
        if np.sum(np.isnan(error1)) + np.sum(np.isnan(error2)) > 0:
            error = np.inf
            print('nan encountered')
        else:
        # Error function to minimize
            error = np.sum(error1) + np.sum(error2)
        errorlst.append(error)


    idx = np.argmin(errorlst)
    #print(errorlst)
    return betalst[idx], alphalst[idx], ISI0lst[idx]


def do_sigma_fitting(subject_file, nsamples, ntrials, sigmalow, sigmahigh, 
                               beta, alpha, ISI0):
    '''Inputs:
    - subject_file: .mat file with the behavioral results of the subject
    - nsamples: number of random (beta,alpha,ISI0) combinations to sample
    - sigmalow, sigmahigh: ranges of sigma for sampling
    
    Outputs:
    sigma: the optimal sigma value to minimize the mse between the std simulated times
    and the mean behavioral times of the subject'''
    sigmalst = np.linspace(sigmalow, sigmahigh, nsamples)

    # Load data for subject SWE(RSG/RSSG, Seth provided)
    swedata = scipy.io.loadmat(subject_file)
    mtp_in = swedata['mtp_in']
    stdtp_in = swedata['stdtp_in']

    target_std1 = stdtp_in[:,0]
    target_std2 = stdtp_in[:,1]

    errorlst = []

    for k in range(nsamples):
        sigmaval = sigmalst[k]

        #1,2,Go simulation
        _, prod_std_lst = simulate_123Go_noise(nstages=2, ntrials=ntrials,
                                    beta=beta, alpha=alpha, ISI0=ISI0, sigma=sigmaval)

        #1,2,3,Go simulation
        _, prod_std_lst2 = simulate_123Go_noise(nstages=3, ntrials=ntrials,
                                    beta=beta, alpha=alpha, ISI0=ISI0, sigma=sigmaval)

        # Continue if there is a nan   
        error1 = (np.transpose(prod_std_lst) * 1000 - target_std1) ** 2
        error2 = (np.transpose(prod_std_lst2) * 1000 - target_std2) ** 2
        
        if np.sum(np.isnan(error1)) + np.sum(np.isnan(error2)) > 0:
            error = np.inf
            print('nan encountered')
        else:
        # Error function to minimize
            error = np.sum(error1) + np.sum(error2)
        errorlst.append(error)


    idx = np.argmin(errorlst)
    #print(errorlst)
    return sigmalst[idx]

def subject_fitting(subject_file, nsamples, nsamples2, ntrials,
                    betalow, betahigh, alphalow, alphahigh, ISI0low, ISI0high,
                    sigmalow, sigmahigh, niter=5, sigma_init=0):
    '''Performs fitting of sigma, I, K for a given subject file,
    Inputs:
    - subject_file: name of .mat behavior file,
    - nsamples: number of I-K combinations to sample
    - nsamples2: number of sigma values to sample
    - ntrials: number of trials for averaging
    - Klow, Khigh, Ilow, Ihigh, sigmalow, sigmahigh: specify the range of K, I and sigma
    - print_every: to print a line every x combinations
    - niter: how many times to alternate between IK sampling and sigma sampling
    - sigma_init: initial value of sigma
    Returns the fitted combination (sigma, I, K)'''
    sigmaval = sigma_init
    for i in range(niter):
        print('Iteration #', i)
        beta, alpha, ISI0 = do_beta_alpha_ISI0_fitting(subject_file, nsamples=nsamples, 
                                                       ntrials=ntrials,
                                               betalow=betalow, betahigh=betahigh, 
                                               alphalow=alphalow, alphahigh=alphahigh, 
                                               ISI0low=ISI0low, ISI0high=ISI0high,
                                              sigma=sigmaval)

        sigmaval = do_sigma_fitting(subject_file, nsamples=nsamples2, ntrials=ntrials,
                        sigmalow=sigmalow, sigmahigh=sigmahigh, 
                                 beta=beta, alpha=alpha, ISI0=ISI0)
        
    # One last round of IK fitting
    beta, alpha, ISI0 = do_beta_alpha_ISI0_fitting(subject_file, nsamples=nsamples, 
                                                       ntrials=ntrials,
                                               betalow=betalow, betahigh=betahigh, 
                                               alphalow=alphalow, alphahigh=alphahigh, 
                                               ISI0low=ISI0low, ISI0high=ISI0high,
                                              sigma=sigmaval)
    return beta, alpha, ISI0, sigma

In [ ]:
dirlst = glob.glob('C:/Users/Le/Dropbox (MIT)/Jazayeri/CircuitModel/Data/*_EKF_*.mat')
np.random.seed(123)
#dirlst = glob.glob('/Users/swe/Dropbox (MIT)/CircuitModel/Data/*_EKF_*.mat')
filename = dirlst[5]
print(filename)
beta, alpha, ISI0 = do_beta_alpha_ISI0_fitting(filename, nsamples=100, ntrials=100,
                                               betalow=0.0, betahigh=0.9, 
                                               alphalow=0, alphahigh=0.01, 
                                               ISI0low=0.500, ISI0high=1.100,
                                              sigma=0)

sigma = do_sigma_fitting(filename, nsamples=100, ntrials=100,
                        sigmalow=0, sigmahigh=0.1, beta=beta, alpha=alpha, ISI0=ISI0)


In [ ]:
beta, alpha, ISI0, sigma = subject_fitting(filename, nsamples=100, nsamples2=100, ntrials=100,
                    betalow=0, betahigh=0.9, alphalow=0, alphahigh=0.01, ISI0low=0.5, ISI0high=1.1,
                    sigmalow=0, sigmahigh=0.1, sigma_init=0, niter=5)

In [ ]:
combi_collection = []
for subject_file in dirlst:
    print(' ********* Doing subject file %s ************' % subject_file)
    combi = subject_fitting(subject_file, nsamples=100, nsamples2=100, ntrials=100,
                betalow=0, betahigh=0.9, alphalow=0, alphahigh=0.01, ISI0low=0.5, ISI0high=1.1,
                sigmalow=0, sigmahigh=0.1, sigma_init=0, niter=5)
    combi_collection.append(combi)
    
combi_arr = np.array(combi_collection)
betas = combi_arr[:,0]
alphas = combi_arr[:,1]
ISI0s = combi_arr[:,2]
sigmas = combi_arr[:,3]

In [ ]:
scipy.io.savemat('discreteAlgorithm_123Go.mat', 
                 {'combi_arr': combi_arr, 'filenames': dirlst})

In [ ]:
id = 1
filename = dirlst[id]
print(filename)
swedata = scipy.io.loadmat(filename)
mtp_in = swedata['mtp_in']
stdtp_in = swedata['stdtp_in']

beta = combi_arr[id, 0]
alpha = combi_arr[id, 1]
ISI0 = combi_arr[id, 2]
sigma = combi_arr[id , 3]

print(alpha,beta,ISI0, sigma)
prod_time_lst, prod_std_lst = simulate_123Go_noise(nstages=2,ntrials=100,
                                      beta=beta,alpha=alpha,ISI0=ISI0, sigma=sigma)
prod_time_lst2, prod_std_lst2 = simulate_123Go_noise(nstages=3,ntrials=100,
                                      beta=beta,alpha=alpha,ISI0=ISI0, sigma=sigma)


plt.subplot(121)
plt.errorbar(ISIs, 1000*prod_time_lst, 1000 * prod_std_lst)
plt.errorbar(ISIs, 1000*prod_time_lst2, 1000 * prod_std_lst2)
#plt.ylim([600, 1000])
plt.title('Model')

plt.subplot(122)
plt.errorbar(ISIs, mtp_in[:,0], stdtp_in[:,0])
plt.errorbar(ISIs, mtp_in[:,1], stdtp_in[:,1])
plt.ylim([600, 1000])
plt.title('Subject')

In [ ]:
# Find subject bias and variance for model
mtp_in_all = []
stdtp_in_all = []
prod_time_lst_all = []
prod_std_lst_all = []
prod_time_lst2_all = []
prod_std_lst2_all = []

for id in range(len(dirlst)):
    filename = dirlst[id]
    print(filename)
    swedata = scipy.io.loadmat(filename)
    mtp_in = swedata['mtp_in']
    stdtp_in = swedata['stdtp_in']

    beta = combi_arr[id, 0]
    alpha = combi_arr[id, 1]
    ISI0 = combi_arr[id, 2]
    sigma = combi_arr[id , 3]

    print(alpha,beta,ISI0, sigma)
    prod_time_lst, prod_std_lst = simulate_123Go_noise(nstages=2,ntrials=100,
                                          beta=beta,alpha=alpha,ISI0=ISI0, sigma=sigma)
    prod_time_lst2, prod_std_lst2 = simulate_123Go_noise(nstages=3,ntrials=100,
                                          beta=beta,alpha=alpha,ISI0=ISI0, sigma=sigma)

    # Aggregate the lists
    mtp_in_all.append(mtp_in)
    stdtp_in_all.append(stdtp_in)
    prod_time_lst_all.append(prod_time_lst)
    prod_std_lst_all.append(prod_std_lst)
    prod_time_lst2_all.append(prod_time_lst2)
    prod_std_lst2_all.append(prod_std_lst2)
    

In [ ]:
scipy.io.savemat('subject_biasvar_discreteAlgo.mat', 
                 {'combi_arr': combi_arr, 'filenames': dirlst,
                'mtp_in_all': mtp_in_all, 'stdtp_in_all': stdtp_in_all,
                'prod_time_lst_all': prod_time_lst_all, 'prod_std_lst_all': prod_std_lst_all,
                'prod_time_lst2_all': prod_time_lst2_all, 'prod_std_lst2_all': prod_std_lst2_all})

# Continuous time algorithm

In [ ]:
def integratorModule(y,I,dt):
    y = y + I*dt
    
    return y

def continuousTimeAlgorithm(ISI,beta,alpha,I0,dt):
    
    totalT = sum(ISI)
    t_s = np.zeros(len(ISI)+1)
    t_s[1:len(ISI)+1] = np.cumsum(ISI)
    t_p = np.zeros(len(ISI)+1)
    t_p_ind = 0
    t_s_ind = 0
    tSteps = int(totalT/dt)
    
    y_s = np.zeros(tSteps+1)
    y_p = np.zeros(tSteps+1)
    I = I0*np.ones(tSteps+1)
    I[0] = I0
    I_p = I
    
    for ti in range(tSteps):
        if t_s[t_s_ind] < ti*dt:
            simulatorError = 1 - y_s[ti]
            I[ti] = I[ti-1] + beta*simulatorError  # Update input into integrator input
            y_s[ti+1] = 0         # Reset simulator on stimulus input
            t_s_ind = t_s_ind+1
        else:
            I[ti] = I[ti-1]
            y_s[ti+1] = integratorModule(y_s[ti],I[ti],dt)
            
        deltaI = alpha*(y_s[ti]-y_p[ti])
        I_p[ti] = I[ti]+deltaI
        
        if y_p[ti] > 1:
            t_p[t_p_ind] = ti*dt
            t_p_ind = t_p_ind+1
            y_p[ti+1] = 0
        else:
            y_p[ti+1] = integratorModule(y_p[ti],I_p[ti],dt)
        
        
    return y_s, y_p, I, I_p, t_p, t_s

In [ ]:
ISI = [0.4] * 50
y_s, y_p, I, I_p, t_p, t_s = continuousTimeAlgorithm(ISI,beta=0.5,alpha=0,I0=1/0.600,dt=0.01)

In [ ]:
IPI = np.diff(t_p)
plt.plot(y_p)